In [2]:
import sys

# use line-buffering for both stdout and stderr
sys.stdout = open(sys.stdout.fileno(), mode="w", buffering=1)
sys.stderr = open(sys.stderr.fileno(), mode="w", buffering=1)

import os
import pathlib
import click
import hydra
import torch
import dill
import wandb
import json
from diffusion_policy.workspace.base_workspace import BaseWorkspace
from omegaconf import OmegaConf
from diffusion_policy.env_runner.robomimic_image_runner import RobomimicImageRunner
import yaml

In [5]:
config_path = r"/home/ubuntu/danny/diffusion_policy/diffusion_policy/config/task/mug_hybrid_cl.yaml"
output_dir = r"/home/ubuntu/danny/diffusion_policy/data/test/test"

In [11]:
# 使用OmegaConf加载配置文件
cfg = OmegaConf.load(config_path)

In [16]:
cfg.dataset_path

['/home/ubuntu/danny/diffusion_policy/data/src/mug_cl/full_demos_50.hdf5', '/home/ubuntu/danny/diffusion_policy/data/src/mug_cl/cl_demos_80.hdf5']

In [20]:
horizon=10
n_obs_steps=2
n_action_steps=8
past_action_visible=False
# 创建环境运行器
env_runner = hydra.utils.instantiate(
    cfg.env_runner,
    output_dir=output_dir,
    dataset_path=cfg.dataset_path[0],  # 使用第一个数据集路径
    max_steps=400,
    env_name=cfg.task_name,
    shape_meta=cfg.shape_meta,
    n_obs_steps=n_obs_steps,
    n_action_steps=n_action_steps,
    past_action=past_action_visible,
)

In [21]:
def eval_policy(ckpt_path, env_runner,device="cuda:0"):
    payload = torch.load(open(ckpt_path, "rb"), pickle_module=dill)
    cfg = payload["cfg"]
    OmegaConf.set_struct(cfg, False)
    cls = hydra.utils.get_class(cfg._target_)
    workspace = cls(cfg, output_dir=output_dir)
    workspace: BaseWorkspace
    workspace.load_payload(payload, exclude_keys=None, include_keys=None)
    # get policy from workspace
    policy = workspace.model
    if cfg.training.use_ema:
        policy = workspace.ema_model
    device = torch.device(device)
    policy.eval()
    return env_runner.run(policy)

In [22]:
runner_log = eval_policy("/home/ubuntu/danny/diffusion_policy/data/outputs/2025.06.05/05.42.51_[50]_finetune/checkpoints/epoch=0070-test_mean_score=0.520.ckpt", env_runner,device="cuda:4")

In [25]:
runner_log

{'train/sim_max_reward_0': 0.0,
 'train/sim_video_0': <wandb.sdk.data_types.video.Video at 0x7f8d25635ac0>,
 'train/sim_max_reward_1': 1.0,
 'train/sim_video_1': <wandb.sdk.data_types.video.Video at 0x7f8d25728730>,
 'train/sim_max_reward_2': 1.0,
 'train/sim_max_reward_3': 1.0,
 'train/sim_max_reward_4': 0.0,
 'train/sim_max_reward_5': 0.0,
 'test/sim_max_reward_100000': 1.0,
 'test/sim_video_100000': <wandb.sdk.data_types.video.Video at 0x7f8d25728640>,
 'test/sim_max_reward_100001': 0.0,
 'test/sim_video_100001': <wandb.sdk.data_types.video.Video at 0x7f8d25728610>,
 'test/sim_max_reward_100002': 1.0,
 'test/sim_video_100002': <wandb.sdk.data_types.video.Video at 0x7f8d257286a0>,
 'test/sim_max_reward_100003': 1.0,
 'test/sim_video_100003': <wandb.sdk.data_types.video.Video at 0x7f8d25728670>,
 'test/sim_max_reward_100004': 0.0,
 'test/sim_max_reward_100005': 1.0,
 'test/sim_max_reward_100006': 1.0,
 'test/sim_max_reward_100007': 1.0,
 'test/sim_max_reward_100008': 1.0,
 'test/sim_m

In [ ]:
cur_output_dir = "./test"
# save runner_log to json
json_log = dict()
for key, value in runner_log.items():
    if isinstance(value, wandb.sdk.data_types.video.Video):
        json_log[key] = value._path
    else:
        json_log[key] = value
out_path = os.path.join(cur_output_dir, "eval_log.json")
json.dump(json_log, open(out_path, "w"), indent=2, sort_keys=True)
